### 중요한 이메일이 오면 윈도우 알람으로 알려주기

In [5]:
import imaplib
import email
from email import policy 

In [6]:
imap = imaplib.IMAP4_SSL('imap.naver.com')
id = 'kktpsh'
pw = 'kitae104@'
imap.login(id, pw)

imap.select('INBOX')
resp, data = imap.uid('search', None, 'All')  # 모든 이메일 검색
all_email = data[0].split()
last_email = all_email[-10:]    # 최근 10개의 이메일만 가져오기 

In [7]:
for mail in reversed(last_email):
    result, data = imap.uid('fetch', mail, '(RFC822)')    # 메일을 읽어온다.
    raw_email = data[0][1]                                # 메일의 내용을 가져온다.
    email_message = email.message_from_bytes(raw_email, policy=policy.default) # 메일의 내용을 파싱한다. 

    print('='*70)
    print('FROM:', email_message['From'])
    print('SENDER:', email_message['Sender'])
    print('TO:', email_message['To'])
    print('DATE:', email_message['Date'])
    print('SUBJECT:', email_message['Subject'])

    print('[CONTENT]')
    message = ''
    if email_message.is_multipart():
        for part in email_message.get_payload():          # 메일의 내용을 가져온다.
            if part.get_content_type() == 'text/plain':   # 메일의 내용 중에서 text/plain 타입만 가져온다.
                bytes = part.get_payload(decode=True)     # 메일의 내용을 가져온다.
                encode = part.get_content_charset()       # 메일의 인코딩을 가져온다.
                message = message + str(bytes, encode)    # 메일의 내용을 문자열로 변환한다.
    print(message)
    print('='*70)

imap.close()
imap.logout()

FROM: 인프런 학습 소식 <study@inflearn.com>
SENDER: None
TO: kktpsh@naver.com
DATE: Wed, 14 Jun 2023 11:48:30 +0000
SUBJECT: [인프런] 지식공유자 김플님이 새소식을 등록했어요.📫
[CONTENT]

FROM: 허유진 <ujin7339@naver.com>
SENDER: None
TO: 김기태교수님 <kktpsh@naver.com>
DATE: Wed, 14 Jun 2023 18:51:31 +0900
SUBJECT: M&A최종본.zip
[CONTENT]

대용량 첨부 1개 48MB

M&A최종본.zip 48MB ~ 2023/07/14
기한이 있는 파일은 30일 보관 / 100회 다운로드 가능




FROM: Google <no-reply@accounts.google.com>
SENDER: None
TO: kktpsh@naver.com
DATE: Sun, 11 Jun 2023 14:28:45 +0000
SUBJECT: aqua0405@gmail.com 관련 보안 경고
[CONTENT]
이 메일은 aqua0405@gmail.com 주소로 전송된 보안 경고의 사본입니다.  
kktpsh@naver.com 주소는 이 계정의
복구 이메일입니다. 모르는 계정이라면 삭제
<https://accounts.google.com/AccountDisavow?adt=AOX8kiqMWYeGCGf3kImk60_f-AezgSi8Klpbe0y6xBy6wq33gkF-STpS2lfc&rfn=325>
하세요
[image: Google]
Windows에서 새로 로그인함


aqua0405@gmail.com
Windows 기기에서 내 Google 계정에 새로 로그인했습니다. 직접 로그인한 것이  
맞다면 아무런 조치를 취하지 않아도 됩니다.
본인이 아니라면 안내에 따라 계정을 보호하세요.
활동 확인
<https://accounts.google.com/AccountChooser?Email=aqua0405@gmail.co

('BYE', [b'Logging out'])